# Caso simplesmente Apoiado

Conforme Timoshenko, a equação da deflexão da placa é computada com a seguinte equação,

\begin{equation}
 w_{max}=\frac{5}{384}\frac{qa^4}{D}-\frac{4qa^4}{\pi^5D}\sum_{m=1,3,5,\cdots}^{\infty}\frac{(-1)^{(m-1)/2}}{m^5}\left(\frac{\alpha_m \tanh \alpha_m + 2}{2 \cosh \alpha_m} \right)
\end{equation}

Nota-se que a equação é denotada como uma soma infinita convergente, então pode-se computar a soma previamente utilizando uma linguagem de programação, no caso o Python. Assim o código abaixo calculará o valor dessa soma.

In [1]:
import sympy as sp
import numpy as np

# Símbolos
a, q, D, b = sp.symbols('a q D b')
m = sp.symbols('m', integer=True)

# Define alpha_m
alpha = (m * sp.pi * b) / (2 * a)

# Termo da série
sign = (-1)**((m - 1)//2)
term1 = 1 / m**5
term2 = (alpha * sp.tanh(alpha) + 2) / (2*sp.cosh(alpha))

summand = sign * term1 * term2

# Valores de lambda
lambda_vals = np.linspace(1, 2.0, 21)
w_normalized = []

# Loop para cada lambda
for λ in lambda_vals:
    summand_l = summand.subs(b, λ * a)
    partial_sum = sum(summand_l.subs(m, mi) for mi in [1, 3, 5, 7, 9])
    num_val = partial_sum.evalf()

    # Cálculo da deflexão
    w = (5/384)*(q*a**4/D)-(4*q*a**4/(sp.pi**5*D))*num_val
    w_norm = w.evalf() * D / (a**4 * q)
    w_normalized.append(w_norm) #finalmente, armazena o valor normalizado



Com o vetor w_not_normalized podemos computar facilmente, substituindo $a,q,D$ pelos mesmos parâmetros do COMSOL teremos 

In [2]:
h, nu, E = sp.symbols('h nu E')

D=(E*h**3)/(12*(1-nu**2))

D = D.subs(E, 25000) # 25000MPa
D = D.subs(h, 0.08) # 8cm de altura da laje
D = D.subs(nu, 0.2) # Coeficiente de poisson do concreto

Factor_evalued=1*1**4/D.evalf()
Factor_evalued

w_real=[]

for i in range(0, len(w_normalized)):
    w_real.append(w_normalized[i]*Factor_evalued*1000)
    print(f"({lambda_vals[i]:,.2f},{w_real[i]:,.5f})")

(1.00,3.65612)
(1.05,4.02083)
(1.10,4.38206)
(1.15,4.73756)
(1.20,5.08548)
(1.25,5.42434)
(1.30,5.75299)
(1.35,6.07056)
(1.40,6.37642)
(1.45,6.67019)
(1.50,6.95162)
(1.55,7.22064)
(1.60,7.47730)
(1.65,7.72175)
(1.70,7.95420)
(1.75,8.17495)
(1.80,8.38432)
(1.85,8.58269)
(1.90,8.77045)
(1.95,8.94802)
(2.00,9.11580)


In [3]:
%reset -f

# Caso biengastado e biapoiado

Conforme Timoshenko, a equação da deflexão da placa é computada com a seguinte equação,

\begin{equation}
 w_{max}=\frac{4qa^4}{\pi^5D}\sum_{m=1,3,5,\cdots}^{\infty}\frac{(-1)^{(m-1)/2}}{m^5}\left( 1 - \frac{\alpha_m \tanh \alpha_m + 2}{2 \cosh \alpha_m} \right)
\end{equation}

Nota-se que a equação é denotada como uma soma infinita convergente, então pode-se computar a soma previamente utilizando uma linguagem de programação, no caso o Python. Assim o código abaixo calculará o valor dessa soma.

In [4]:
import sympy as sp
import numpy as np

# Símbolos
a, q, D, b = sp.symbols('a q D b')
m = sp.symbols('m', integer=True)

# Define alpha_m
alpha = (m * sp.pi * b) / (2 * a)

# Termo da série
sign = (-1)**((m - 1)//2)
term1 = 1 / m**5
term2 = alpha * sp.tanh(alpha) / sp.cosh(alpha)
numerator = alpha - sp.tanh(alpha) * (1 + alpha * sp.tanh(alpha))
denominator = alpha - sp.tanh(alpha) * (alpha * sp.tanh(alpha) - 1)
term3 = numerator / denominator

summand = sign * term1 * term2 * term3

# Valores de lambda
lambda_vals = np.linspace(1, 2.0, 21)
w_normalized = []

# Loop para cada lambda
for λ in lambda_vals:
    summand_l = summand.subs(b, λ * a)
    partial_sum = sum(summand_l.subs(m, mi) for mi in [1, 3, 5, 7, 9])
    num_val = partial_sum.evalf()

    # Cálculo da deflexão
    w = (5 / 384 - (2 / sp.pi) * num_val) * (q * a**4 / D)
    w_norm = w.evalf() * D / (a**4 * q)
    w_normalized.append(w_norm) #finalmente, armazena o valor normalizado

In [5]:
w_normalized

[0.221984244378667,
 0.232518467341625,
 0.241070358457379,
 0.247638195877344,
 0.252274202237558,
 0.255073482131707,
 0.256163208518176,
 0.255692618988855,
 0.253824180744559,
 0.250726111634408,
 0.246566308730513,
 0.241507635884912,
 0.235704454543048,
 0.229300242823847,
 0.222426130748573,
 0.215200178736670,
 0.207727236861180,
 0.200099239490867,
 0.192395810415061,
 0.184685074847226,
 0.177024595141214]